In [3]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

traindata=traindata.drop('id',axis=1)
testdata=testdata.drop('id',axis=1)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])
traindata['attack_cat'] = lb_make.fit_transform(traindata['attack_cat'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])
testdata['attack_cat'] = lb_make.fit_transform(testdata['attack_cat'])

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-2]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-2]

In [4]:
Arr = X.values
arr = x.values

import librosa
Arr_Delta = librosa.feature.delta(Arr)
arr_delta = librosa.feature.delta(arr)

Delta1 = pd.DataFrame(Arr_Delta)
delta1 = pd.DataFrame(arr_delta)

scaler = Normalizer().fit(Delta1)
Delta1 = scaler.transform(Delta1)
scaler = Normalizer().fit(delta1)
delta1 = scaler.transform(delta1)

Delta1 = np.reshape(Delta1, (Delta1.shape[0],Delta1.shape[1],1))
delta1 = np.reshape(delta1, (delta1.shape[0],delta1.shape[1],1))

scaler = Normalizer().fit(X)
X = scaler.transform(X)
scaler = Normalizer().fit(x)
x = scaler.transform(x)

Y = np.array(Y)
y = np.array(y)

X = np.reshape(X, (X.shape[0],X.shape[1],1))
x = np.reshape(x, (x.shape[0],x.shape[1],1))
lstm_output_size = 70

Y = to_categorical(Y)
y = to_categorical(y)

In [5]:
from keras.layers import Input, Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, concatenate, Reshape
from keras.models import Model

lstm_output_size=70
Xshape = Input(shape=(41, 1))
Delta1shape = Input(shape=(41, 1))

x1 = Convolution1D(64, 3, border_mode="same",activation="relu")(Xshape)
x1 = MaxPooling1D(pool_length=(2))(x1)
x1 = Dropout(0.5)(x1)
   
x2 = Convolution1D(64, 3, border_mode="same",activation="relu")(Delta1shape)
x2 = MaxPooling1D(pool_length=(2))(x2)
x2 = Dropout(0.5)(x2)

output = concatenate([x1, x2])

x3 = Bidirectional(LSTM(lstm_output_size))(output)
x3 = Dropout(0.1)(x3)
x3 = Dense(10, activation="softmax")(x3)

model = Model([Xshape, Delta1shape],x3)
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
print(model.summary())

/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  if sys.path[0] == '':
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 41, 64)       256         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 41, 64)       256         input_2[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.fit([X,Delta1], Y, epochs=200, validation_data=([x,delta1], y))

/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu")`
  
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu")`
  if sys.path[0] == '':
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 41, 64)       256         input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 41, 64)       256         input_6[0][0]                    
____________________________________________________________________________________________

Epoch 89/200
175341/175341 [==============================] - 315s 2ms/step - loss: 0.8954 - accuracy: 0.6504 - val_loss: 0.8708 - val_accuracy: 0.6509
Epoch 90/200
175341/175341 [==============================] - 320s 2ms/step - loss: 0.8956 - accuracy: 0.6498 - val_loss: 0.8712 - val_accuracy: 0.6287
Epoch 91/200
175341/175341 [==============================] - 315s 2ms/step - loss: 0.8921 - accuracy: 0.6512 - val_loss: 0.8675 - val_accuracy: 0.6213
Epoch 92/200
175341/175341 [==============================] - 315s 2ms/step - loss: 0.8935 - accuracy: 0.6505 - val_loss: 0.8910 - val_accuracy: 0.6388
Epoch 93/200
175341/175341 [==============================] - 316s 2ms/step - loss: 0.8943 - accuracy: 0.6508 - val_loss: 0.8822 - val_accuracy: 0.6078
Epoch 94/200
175341/175341 [==============================] - 319s 2ms/step - loss: 0.8923 - accuracy: 0.6508 - val_loss: 0.8817 - val_accuracy: 0.6375
Epoch 95/200
175341/175341 [==============================] - 317s 2ms/step - loss: 0.89

In [ ]:
from sklearn.metrics import accuracy_score
predictions = model.predict([x,delta1])

import sys
np.set_printoptions(threshold=sys.maxsize)
pred = np.zeros((82332,1), dtype=int)
for i in range(0, predictions.shape[0]):
    pred[i] = np.where(predictions[i] == np.amax(predictions[i]))

pred = to_categorical(pred)
z = np.zeros((82332,3), dtype=int)
pred = np.append(pred, z, axis=1)

pred.shape

In [ ]:
print("Accuracy: ",accuracy_score(y,pred))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix,classification_report,confusion_matrix
print("Classification Report: \n",classification_report(y,pred))
conmat = np.array(confusion_matrix(y.argmax(axis=1), pred.argmax(axis=1)))
confusion = pd.DataFrame(conmat, index=['0','1','2','3','4','5','6','7','8','9'],
                        columns=[['0','1','2','3','4','5','6','7','8','9']])
print (conmat)
conmat.shape

In [7]:
from keras.layers import Input, Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, concatenate, Reshape
from keras.models import Model
from keras.regularizers import l2

lstm_output_size=70
Xshape = Input(shape=(41, 1))
Delta1shape = Input(shape=(41, 1))

x1 = Convolution1D(64, 3, border_mode="same",activation="relu")(Xshape)
x1 = MaxPooling1D(pool_length=(2))(x1)
x1 = Dropout(0.5)(x1)
   
x2 = Convolution1D(64, 3, border_mode="same",activation="relu")(Delta1shape)
x2 = MaxPooling1D(pool_length=(2))(x2)
x2 = Dropout(0.5)(x2)

output = concatenate([x1, x2])

x3 = Bidirectional(LSTM(lstm_output_size, kernel_regularizer = l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer = l2(0.01)))(output)
x3 = Dropout(0.1)(x3)
x3 = Dense(10, activation="softmax")(x3)

model = Model([Xshape, Delta1shape],x3)
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
print(model.summary())
history = model.fit([X,Delta1], Y, epochs=100, validation_data=([x,delta1], y))

/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  if __name__ == '__main__':
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  # Remove the CWD from sys.path while we load stuff.
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  del sys.path[0]
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 41, 1)        0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 41, 64)       256         input_9[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 41, 64)       256         input_10[0][0]                   
____________________________________________________________________________________________

KeyboardInterrupt: 

In [4]:
from keras.layers import Input, Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, concatenate, Reshape
from keras.models import Model
from keras.regularizers import l2, l1

lstm_output_size=70
Xshape = Input(shape=(41, 1))
Delta1shape = Input(shape=(41, 1))

x1 = Convolution1D(64, 3, border_mode="same",activation="relu")(Xshape)
x1 = MaxPooling1D(pool_length=(2))(x1)
x1 = Dropout(0.5)(x1)
   
x2 = Convolution1D(64, 3, border_mode="same",activation="relu")(Delta1shape)
x2 = MaxPooling1D(pool_length=(2))(x2)
x2 = Dropout(0.5)(x2)

output = concatenate([x1, x2])

x3 = Bidirectional(LSTM(lstm_output_size))(output)
x3 = Dropout(0.1)(x3)
x3 = Dense(10, activation="softmax")(x3)

model = Model([Xshape, Delta1shape],x3)
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
print(model.summary())
history = model.fit([X,Delta1], Y, epochs=100, validation_data=([x,delta1], y))

/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  if __name__ == '__main__':
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  # Remove the CWD from sys.path while we load stuff.
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  del sys.path[0]
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 41, 64)       256         input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 41, 64)       256         input_4[0][0]                    
____________________________________________________________________________________________

Epoch 89/100
175341/175341 [==============================] - 289s 2ms/step - loss: 0.8875 - accuracy: 0.6515 - val_loss: 0.8605 - val_accuracy: 0.6087
Epoch 90/100
175341/175341 [==============================] - 290s 2ms/step - loss: 0.8890 - accuracy: 0.6517 - val_loss: 0.8452 - val_accuracy: 0.6369
Epoch 91/100
175341/175341 [==============================] - 289s 2ms/step - loss: 0.8857 - accuracy: 0.6522 - val_loss: 0.8849 - val_accuracy: 0.6210
Epoch 92/100
175341/175341 [==============================] - 288s 2ms/step - loss: 0.8874 - accuracy: 0.6515 - val_loss: 0.8720 - val_accuracy: 0.6237
Epoch 93/100
175341/175341 [==============================] - 290s 2ms/step - loss: 0.8861 - accuracy: 0.6528 - val_loss: 0.8712 - val_accuracy: 0.6250
Epoch 94/100
175341/175341 [==============================] - 289s 2ms/step - loss: 0.8899 - accuracy: 0.6521 - val_loss: 0.8638 - val_accuracy: 0.6258
Epoch 95/100
175341/175341 [==============================] - 290s 2ms/step - loss: 0.88

In [9]:
from sklearn.metrics import accuracy_score
predictions = model.predict([x,delta1])

import sys
np.set_printoptions(threshold=sys.maxsize)
pred = np.zeros((82332,1), dtype=int)
for i in range(0, predictions.shape[0]):
    pred[i] = np.where(predictions[i] == np.amax(predictions[i]))
pred = to_categorical(pred)
pred.shape

(82332, 8)

In [10]:
z = np.zeros((82332,2), dtype=int)
pred = np.append(pred, z, axis=1)

pred.shape


print("Accuracy: ",accuracy_score(y,pred))

from sklearn.metrics import multilabel_confusion_matrix,classification_report,confusion_matrix
print("Classification Report: \n",classification_report(y,pred))
conmat = np.array(confusion_matrix(y.argmax(axis=1), pred.argmax(axis=1)))
confusion = pd.DataFrame(conmat, index=['0','1','2','3','4','5','6','7','8','9'],
                        columns=[['0','1','2','3','4','5','6','7','8','9']])
print (conmat)
conmat.shape

Accuracy:  0.6210222027887091


/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       677
           1       1.00      0.00      0.00       583
           2       0.35      0.01      0.01      4089
           3       0.27      0.78      0.40     11132
           4       0.23      0.15      0.18      6062
           5       1.00      0.96      0.98     18871
           6       0.85      0.60      0.70     37000
           7       0.74      0.37      0.49      3496
           8       0.00      0.00      0.00       378
           9       0.00      0.00      0.00        44

   micro avg       0.62      0.62      0.62     82332
   macro avg       0.44      0.29      0.28     82332
weighted avg       0.72      0.62      0.63     82332
 samples avg       0.62      0.62      0.62     82332

[[    0     0     3   630     1     0    10    33     0     0]
 [    0     1     6   528     2     0    13    33     0     0]
 [    0     0    23  3478   134    1

(10, 10)

In [7]:
history = model.fit([X,Delta1], Y, epochs=200, validation_data=([x,delta1], y))

Train on 175341 samples, validate on 82332 samples
Epoch 1/200
175341/175341 [==============================] - 181s 1ms/step - loss: 1.3443 - accuracy: 0.5405 - val_loss: 1.2772 - val_accuracy: 0.5997
Epoch 2/200
175341/175341 [==============================] - 168s 959us/step - loss: 1.3282 - accuracy: 0.5460 - val_loss: 1.2803 - val_accuracy: 0.5933
Epoch 3/200
175341/175341 [==============================] - 169s 963us/step - loss: 1.3220 - accuracy: 0.5477 - val_loss: 1.2852 - val_accuracy: 0.5870
Epoch 4/200
175341/175341 [==============================] - 191s 1ms/step - loss: 1.3181 - accuracy: 0.5491 - val_loss: 1.2878 - val_accuracy: 0.5891
Epoch 5/200
175341/175341 [==============================] - 185s 1ms/step - loss: 1.3152 - accuracy: 0.5501 - val_loss: 1.2773 - val_accuracy: 0.5873
Epoch 6/200
175341/175341 [==============================] - 196s 1ms/step - loss: 1.2621 - accuracy: 0.5568 - val_loss: 1.1190 - val_accuracy: 0.6173
Epoch 7/200
175341/175341 [============

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-b3afaf624b5d>", line 1, in <module>
    history = model.fit([X,Delta1], Y, epochs=200, validation_data=([x,delta1], y))
  File "/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/keras/engine/training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 196, in fit_loop
    outs = fit_function(ins_batch)
  File "/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/tensorflow/python/keras/backend.py", line 3251, in __call__
    inputs = nest.flatten(inputs)
  File "/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/tensorflow/python/util/nest.py", line 241, in flatten
    return _pywrap_tensorflow.Flatten

KeyboardInterrupt: 